In [1]:
from pythtb import Lattice, Mesh, TBModel, WFArray
import numpy as np

In [2]:
lattice = Lattice(
    lat_vecs=[[1.0]], orb_vecs=[[0.0], [1 / 3], [2 / 3]], periodic_dirs=[0]
)

In [3]:
mesh = Mesh(dim_k=1, axis_types=["k"])

In [4]:
mesh.build_grid([20])

In [5]:
wfa = WFArray(lattice=lattice, mesh=mesh)

In [6]:
def set_model(t: float, delta: float, lmbda: float) -> TBModel:
    """Periodic three-site model at parameter lmbda."""

    model = TBModel(lattice=lattice)

    # nearest-neighbour hoppings (last hop wraps to the next cell)
    model.set_hop(t, 0, 1, [0])
    model.set_hop(t, 1, 2, [0])
    model.set_hop(t, 2, 0, [1])

    onsite = [delta * -np.cos(2 * np.pi * (lmbda - idx / 3)) for idx in range(3)]
    model.set_onsite(onsite)
    return model


model = set_model(t=1.0, delta=1.0, lmbda=0.0)
wfa.solve_model(model)

In [7]:
wfa[2]

array([[ 0.87598308-0.j        , -0.2806304 +0.1938386j ,
        -0.2806304 -0.1938386j ],
       [ 0.26138584+0.j        , -0.06155942-0.67974198j,
        -0.06155942+0.67974198j],
       [ 0.40537771+0.j        ,  0.64610914+0.0194278j ,
         0.64610914-0.0194278j ]])

In [8]:
t = -1.3
delta = 2.0

model = TBModel(lattice=lattice)

# nearest-neighbour hoppings (last hop wraps to the next cell)
model.set_hop(t, 0, 1, [0])
model.set_hop(t, 1, 2, [0])
model.set_hop(t, 2, 0, [1])

# Per-orbital onsite as lambdas of lmbda
onsite = [
    lambda lmbda: delta * -np.cos(2 * np.pi * (lmbda - 0 / 3)),
    lambda lmbda: delta * -np.cos(2 * np.pi * (lmbda - 1 / 3)),
    lambda lmbda: delta * -np.cos(2 * np.pi * (lmbda - 2 / 3)),
]

model.set_onsite(onsite)
print(model)

----------------------------------------
       Tight-binding model report       
----------------------------------------
r-space dimension           = 1
k-space dimension           = 1
periodic directions         = [0]
spinful                     = False
number of spin components   = 1
number of electronic states = 3
number of orbitals          = 3

Lattice vectors (Cartesian):
  # 0 ===> [ 1.000]
Volume of unit cell (Cartesian) = 1.000 [A^d]

Reciprocal lattice vectors (Cartesian):
  # 0 ===> [ 6.283]
Volume of reciprocal unit cell = 6.283 [A^-d]

Orbital vectors (Cartesian):
  # 0 ===> [ 0.000]
  # 1 ===> [ 0.333]
  # 2 ===> [ 0.667]

Orbital vectors (fractional):
  # 0 ===> [ 0.000]
  # 1 ===> [ 0.333]
  # 2 ===> [ 0.667]
----------------------------------------
Site energies:
  < 0 | H | 0 > = lambda lmbda: delta * -np.cos(2 * np.pi * (lmbda - 0 / 3)),
  < 1 | H | 1 > = lambda lmbda: delta * -np.cos(2 * np.pi * (lmbda - 1 / 3)),
  < 2 | H | 2 > = lambda lmbda: delta * -np.cos(2 *

In [9]:
mesh = Mesh(
    dim_k=1,
    dim_lambda=1,
    axis_types=["k", "l"],  # first axis: crystal momentum; second: adiabatic parameter
    axis_names=["kx", "lmbda"],
)

In [10]:
mesh.build_grid(shape=(31, 21), gamma_centered=True, lambda_start=0.0, lambda_stop=1.0)

In [11]:
mesh.loop(
    axis_idx=1, component_idx=1, closed=True
)  # form the lambda axis into a closed loop
print(mesh)

Mesh Summary
Type: grid
Dimensionality: 1 k-dim(s) + 1 λ-dim(s)
Number of mesh points: 651
Full shape: (31, 21, 2)
k-axes: [Axis(type=k, name=kx, size=31)]
λ-axes: [Axis(type=l, name=lmbda, size=21)]
Is a torus in k-space (all k-axes wind BZ): yes
Loops: (axis 0, comp 0, winds_bz=yes, closed=no), (axis 1, comp 1, winds_bz=no, closed=yes)


In [12]:
wfa = WFArray(lattice, mesh)

In [13]:
kpts = np.linspace(-0.5, 0.5, 21)[:, None]
H = model.hamiltonian(kpts, lmbda=np.linspace(0, 1, 10, endpoint=True))
H.shape

(21, 10, 3, 3)

In [14]:
wfa.solve_model(model=model)

In [15]:
np.allclose(wfa[0, 0], wfa[0, -1])  # first k-point, first and last lambda points

True